In [2]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import csv

In [4]:
# Credenciales de Twitter
USERNAME = 'ScrapeoUPM'
PASSWORD = 'scrapeafutbol'
EMAIL = 'oscar.marin@alumnos.upm.es'

SCROLL_PAUSE_TIME = 5  # Tiempo de espera para que carguen nuevos tweets

In [5]:
def login(driver, username, email, password):
    
    # Navegar a la página de inicio de sesión de Twitter
    driver.get('https://twitter.com/login')
    time.sleep(7)  # Esperar a que cargue la página

    # Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN) #Presionar tecla enter
    time.sleep(2)
    
    try:
        # Buscar el campo de confirmación de correo
        email_input = driver.find_element(By.NAME, 'text')
        email_input.send_keys(email)
        email_input.send_keys(Keys.RETURN)
        time.sleep(2)
    except:
        pass
    
    # Buscar el campo de contraseña e ingresar la contraseña
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)

def scrap_n_tweets(driver, n_tweets: int, word_search: str, date_range: tuple):

    word_link = word_search.replace(' ', '%20')
    date_range = f'%20until%3A{date_range[1]}%20since%3A{date_range[0]}'

    # Navegar a la página de búsqueda de Twitter
    driver.get(f'https://twitter.com/search?q={word_link}{date_range}')
    time.sleep(5)  # Esperar a que cargue la página

    n = 0
    tweets = []
    while n < n_tweets:
        # Extraer los tweets visibles en este momento
        new_tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        # Imprimir el texto de los tweets extraídos
        for tweet in new_tweets:
            tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
            tweet_date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            tweet_lang  = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]').get_attribute('lang')
            tweets.append([word_search, tweet_text.text, tweet_date, tweet_lang])
            n += 1
            if n == n_tweets:
                break

        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

    return tweets

def scrap_all_tweets(driver, word_search: str, date_range: tuple):
    
    word_link = word_search.replace(' ', '%20')
    date_range = f'%20until%3A{date_range[1]}%20since%3A{date_range[0]}'

    # Navegar a la página de búsqueda de Twitter
    driver.get(f'https://twitter.com/search?q={word_link}{date_range}+-filter%3Areplies&src=typed_query&f=top')
    time.sleep(5)  # Esperar a que cargue la página
  
    last_height = driver.execute_script("return document.body.scrollHeight")
    tweets = []
    while True:
        # Extraer los tweets visibles en este momento
        new_tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        
        # Imprimir el texto de los tweets extraídos
        for tweet in new_tweets:
            tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
            tweet_date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute('datetime')
            tweet_lang  = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]').get_attribute('lang')
            tweets.append([word_search, tweet_text.text, tweet_date, tweet_lang])
            
        # Hacer scroll hasta el final de la página
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    

    return tweets

In [6]:
def get_weeks(since, until):
    weeks = []
    actual_date = datetime.strptime(since, '%Y-%m-%d')
    final_date = datetime.strptime(until, '%Y-%m-%d')

    while actual_date + timedelta(days=7) < final_date:
        # Definir el rango de cada semana
        week_end = actual_date + timedelta(days=6) # Día actual + 6 días = 7 días

        weeks.append((actual_date.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))
        # Avanzar a la siguiente semana
        actual_date = week_end + timedelta(days=1) # Empieza el siguiente día para no repetir

    weeks.append((actual_date.strftime('%Y-%m-%d'), until))
    return weeks

get_weeks('2024-09-30', '2024-11-06')

[('2024-09-30', '2024-10-06'),
 ('2024-10-07', '2024-10-13'),
 ('2024-10-14', '2024-10-20'),
 ('2024-10-21', '2024-10-27'),
 ('2024-10-28', '2024-11-03'),
 ('2024-11-04', '2024-11-06')]

In [7]:
def save_data(data, filename):
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')  
        for row in data:
            writer.writerow(row)

In [9]:
# Si es la primera ejecución del jugador

player = 'Ferran Torres'
since = '2023-08-01'
until = '2024-07-15'
n_tweets = 100

chrome_options = Options()
#chrome_options.add_argument("--headless")  # Ejecuta Chrome en modo headless
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
login(driver, USERNAME, EMAIL, PASSWORD)

weeks = get_weeks(since, until)

# Crear un archivo CSV con las columnas correspondientes
with open(f'raw/tweets/{player.replace(' ', '_')}_tweets.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(['player', 'text', 'date', 'lang'])

# Iterar sobre las semanas y extraer los tweets
all_tweets = []
for i, (since, until) in enumerate(weeks):
    print(f'Scraping tweets from {since} to {until}')
    tweets = scrap_all_tweets(driver, player, (since, until))
    print(f'Found {len(tweets)} tweets')
    if len(tweets) == 0:
        print(f'Failed in recollection from {(since, until)}')
        save_data(all_tweets, f'raw/tweets/{player.replace(' ', '_')}_tweets.csv')
        break
    
    all_tweets.extend(tweets)
    time.sleep(5)


# Cerrar el navegador
driver.quit()
tweets

SyntaxError: f-string: unmatched '(' (2623381166.py, line 18)

In [10]:
# Si quieres reanudar con un jugador que ya tiene tweets recopilados

player = 'Ferran Torres'
since = '2023-08-01'
until = '2024-07-15'
n_tweets = 100

chrome_options = Options()
#chrome_options.add_argument("--headless")  # Ejecuta Chrome en modo headless
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
login(driver, USERNAME, EMAIL, PASSWORD)

weeks = get_weeks(since, until)
# Ver en qué semana se quedó
ini = weeks.index(('2024-01-30', '2024-02-05')) # Poner manualmente

# Iterar sobre las semanas y extraer los tweets
all_tweets = []
for i, (since, until) in enumerate(weeks[ini:]):
    print(f'Scraping tweets from {since} to {until}')
    tweets = scrap_all_tweets(driver, player, (since, until))
    print(f'Found {len(tweets)} tweets')
    if len(tweets) == 0:
        print(f'Failed in recollection from {(since, until)}')
        save_data(all_tweets, f'raw/tweets/{player.replace(' ', '_')}_tweets.csv')
        break
    
    all_tweets.extend(tweets)
    time.sleep(5)


# Cerrar el navegador
driver.quit()
tweets

SyntaxError: f-string: unmatched '(' (3123321355.py, line 27)

In [45]:
weeks

[('2023-08-01', '2023-08-07'),
 ('2023-08-08', '2023-08-14'),
 ('2023-08-15', '2023-08-21'),
 ('2023-08-22', '2023-08-28'),
 ('2023-08-29', '2023-09-04'),
 ('2023-09-05', '2023-09-11'),
 ('2023-09-12', '2023-09-18'),
 ('2023-09-19', '2023-09-25'),
 ('2023-09-26', '2023-10-02'),
 ('2023-10-03', '2023-10-09'),
 ('2023-10-10', '2023-10-16'),
 ('2023-10-17', '2023-10-23'),
 ('2023-10-24', '2023-10-30'),
 ('2023-10-31', '2023-11-06'),
 ('2023-11-07', '2023-11-13'),
 ('2023-11-14', '2023-11-20'),
 ('2023-11-21', '2023-11-27'),
 ('2023-11-28', '2023-12-04'),
 ('2023-12-05', '2023-12-11'),
 ('2023-12-12', '2023-12-18'),
 ('2023-12-19', '2023-12-25'),
 ('2023-12-26', '2024-01-01'),
 ('2024-01-02', '2024-01-08'),
 ('2024-01-09', '2024-01-15'),
 ('2024-01-16', '2024-01-22'),
 ('2024-01-23', '2024-01-29'),
 ('2024-01-30', '2024-02-05'),
 ('2024-02-06', '2024-02-12'),
 ('2024-02-13', '2024-02-19'),
 ('2024-02-20', '2024-02-26'),
 ('2024-02-27', '2024-03-04'),
 ('2024-03-05', '2024-03-11'),
 ('2024-

In [42]:
import pandas as pd

df = pd.read_csv(f'raw/tweets/{player.replace(' ', '_')}_tweets.csv', delimiter=';')
df

,player,text,date,lang
0,Ferran Torres,"between jana, pina and bruna also being so you...",2023-08-01T21:01:13.000Z,en
1,Ferran Torres,So… you’ve met Freminet’s friend Pers,2023-08-04T16:53:31.000Z,en
2,Ferran Torres,ANDREW FARRELL SENDS THE \r\n@NEREVOLUTION\r\n...,2023-08-04T02:37:21.000Z,en
3,Ferran Torres,MAINSTREAM vs QUEEN OF ALT vs MEL-LOCAL oh fe...,2023-08-01T02:31:46.000Z,en
4,Ferran Torres,Chin's Ig Post - Francine's favorite place !!...,2023-08-04T04:09:47.000Z,en
...,...,...,...,...
3928,Ferran Torres,Ferran Torres misses his birthday 75% of all y...,2024-02-29T16:37:09.000Z,en
3929,Ferran Torres,• Vinicius Jr a encvlé Ansu Fati\n• Vinicius J...,2024-03-03T20:06:41.000Z,fr
3930,Ferran Torres,Empieza el entreno del Barça para preparar el ...,2024-03-02T10:15:25.000Z,es
3931,Ferran Torres,mucho se habla de lamine pero poco de que tene...,2024-02-29T14:43:49.000Z,es
